# What's cooking?

Tensorflow MLPClassifier com camada de Embedding

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import sklearn as sk
train_df = pd.read_json('./train.json')
test_df = pd.read_json('./test.json')

train_df.sample(5)

In [ ]:
# # coletando ingredientes:
# ingredients = train_df.ingredients.str.join(' ')
# # ingredients
# # removendo caracteres especiais, stemming e lemmatizing:
# import nltk
# from nltk.stem import WordNetLemmatizer
# from nltk import word_tokenize
# nltk.download('wordnet')
# nltk.download('omw-1.4')
# nltk.download('punkt')
# lemmatizer = WordNetLemmatizer()

# def lemmatize_text(text):
#     return [lemmatizer.lemmatize(w) for w in word_tokenize(text)]

train = train_df.copy()
train['X'] = train_df.ingredients.str.join(' ')
# train['X'].apply(lemmatize_text)

train.head()

In [ ]:
# encode (label) targets:
from sklearn.preprocessing import LabelEncoder
label_encoder = LabelEncoder()
cuisine = label_encoder.fit_transform(train_df['cuisine'].values)
cuisine

In [ ]:
# join data
y = cuisine.copy()
X = train.X.copy()
X.shape

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)
print(X_train.shape)
print(y_train.shape)

In [ ]:
# Imports
import tensorflow as tf

print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))
sess = tf.compat.v1.Session(config=tf.compat.v1.ConfigProto(log_device_placement=True))

In [ ]:
%%time
from keras.preprocessing import sequence, text

X_train, X_valid, y_train, y_valid = train_test_split(X_train, y_train, test_size=0.15, random_state=42)

token = text.Tokenizer(num_words=None)
max_len = 80
# len(train.X.max())

token.fit_on_texts(list(X_train) + list(X_valid))
xtrain_seq = token.texts_to_sequences(X_train)
xvalid_seq = token.texts_to_sequences(X_valid)

#zero pad the sequences
xtrain_pad = sequence.pad_sequences(xtrain_seq, maxlen=max_len)
xvalid_pad = sequence.pad_sequences(xvalid_seq, maxlen=max_len)

word_index = token.word_index
word_index

In [ ]:
from tqdm.notebook import tqdm

# load the GloVe vectors in a dictionary:
# http://www-nlp.stanford.edu/data/glove.840B.300d.zip

embeddings_index = {}
f = open('./glove.6B.300d.txt','r',encoding='utf-8')
for line in tqdm(f):
    values = line.split(' ')
    word = values[0]
    coefs = np.asarray([float(val) for val in values[1:]])
    embeddings_index[word] = coefs
f.close()

print('Found %s word vectors.' % len(embeddings_index))

# create an embedding matrix for the words we have in the dataset
embedding_matrix = np.zeros((len(word_index) + 1, 300))
for word, i in tqdm(word_index.items()):
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        embedding_matrix[i] = embedding_vector

In [ ]:
%%time
from keras.layers import SimpleRNN
from keras.layers import Embedding
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import SpatialDropout1D
from keras.layers.recurrent import GRU

# # A simpleRNN without any pretrained embeddings and one dense layer
# model = Sequential(name = 'SimpleRNN')
# model.add(Embedding(len(word_index) + 1,
#                     300,
#                     input_length=max_len))
# model.add(SimpleRNN(100))
# model.add(Dense(1, activation='softmax'))

# # binary_crossentropy
# # categorical_crossentropy

# # GRU with glove embeddings and two dense layers
model = Sequential(name= 'GRU')
model.add(Embedding(len(word_index) + 1,
                300,
                weights=[embedding_matrix],
                input_length=max_len,
                trainable=False))
model.add(SpatialDropout1D(0.3))
model.add(GRU(300))
model.add(Dense(1, activation='softmax'))

model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
model.summary()


In [ ]:
%%time
model.fit(xtrain_pad, y_train, 
    # epochs=5,
    # batch_size = 64*2
    ) #, nb_epoch=5)# , batch_size=64*strategy.num_replicas_in_sync)

In [ ]:
xtest_seq = token.texts_to_sequences(X_test)
xtest_pad = sequence.pad_sequences(xtest_seq, maxlen=max_len)
test_results = model.evaluate(x=xtest_pad,y= y_test, verbose=1)
print(f'Test results - Loss: {test_results[0]} - Accuracy: {test_results[1]}%')